In [1]:
# import libraries we need
%pylab inline
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
tfd = tfp.distributions

import sys
sys.path.append('./../py_programs/')
from Simulator import *
from MakeModel import *
from keract import get_activations

Populating the interactive namespace from numpy and matplotlib


### load data from files

In [2]:
# load the data from files
filenames = []
x_ = []
y_ = []

# iterate over all files
#filenames = ['./../simulation/data/laser'] + ['./../simulation/data/sps']
filenames = ['./../simulation/data/halfhalf_rand/'+str(i) for i in range(50)]


for filename in filenames: 
    data = load_data(filename=filename)
    x_.append(data)
    
x_ = np.array(x_)

In [3]:
# convert data into correct shape
x_ = x_.reshape(x_.shape[0]*x_.shape[1],x_.shape[2],x_.shape[3])

# get the ground truth
threshold = 0.5
y_ = get_truth(x_, thr=threshold)

# we only need the g2signal for training
x_ = x_[:,2,:]

Now we've collected all the datasets. For training the model, we need to **shuffle** all data sets first


In [4]:
# shuffle
i = np.random.permutation(len(x_))
x_ = x_[i]
y_ = y_[i]

# reshape
x_ = x_.reshape(x_.shape[0], x_.shape[1], 1)
y_ = y_.reshape(y_.shape[0])

# allocate
x_train = x_[100:]
y_train = y_[100:]
x_test = x_[:100]
y_test = y_[:100]

In [5]:
# check the sps rate (data sets considered as sps out of all data sets) 
y_[y_==1].shape[0]/y_.shape[0]

0.585

---
# CNN model

In [6]:
# create a model
model = make_model(x_train.shape[1:])


In [7]:
epochs = 30
batch_size = 5

callbacks = set_callback(save=True,name='model_3')
model_compile(model)

<bound method Model.compile of <tensorflow.python.keras.engine.functional.Functional object at 0x7fc64e0144f0>>

In [8]:
model_train(model, x_train,y_train, batch_size=5, epochs=30, callbacks=callbacks)

Epoch 1/30
686/686 [==============================] - 1s 856us/step - loss: 0.1215 - sparse_categorical_accuracy: 0.4134 - binary_accuracy: 0.8398 - val_loss: 0.0311 - val_sparse_categorical_accuracy: 0.4299 - val_binary_accuracy: 0.9830
Epoch 2/30
686/686 [==============================] - 0s 543us/step - loss: 0.0227 - sparse_categorical_accuracy: 0.4008 - binary_accuracy: 0.9830 - val_loss: 0.0156 - val_sparse_categorical_accuracy: 0.4299 - val_binary_accuracy: 0.9810
Epoch 3/30
686/686 [==============================] - 0s 544us/step - loss: 0.0145 - sparse_categorical_accuracy: 0.4194 - binary_accuracy: 0.9866 - val_loss: 0.0140 - val_sparse_categorical_accuracy: 0.4299 - val_binary_accuracy: 0.9803
Epoch 4/30
686/686 [==============================] - 0s 555us/step - loss: 0.0113 - sparse_categorical_accuracy: 0.4192 - binary_accuracy: 0.9884 - val_loss: 0.0108 - val_sparse_categorical_accuracy: 0.4299 - val_binary_accuracy: 0.9844
Epoch 5/30
686/686 [============================

In [ ]:
visualize_layer(model,x_train)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 1)]          0         
_________________________________________________________________
lambda (Lambda)              (None, 200, 1)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 40, 1)             0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 40, 1)             0         
_________________________________________________________________
conv1d (Conv1D)              (None, 40, 4)             16        
_________________________________________________________________
batch_normalization (BatchNo (None, 40, 4)             16        
_________________________________________________________________
re_lu (ReLU)                 (None, 40, 4)             0     

We print out the first 15 values to see how accurate the model is:

In [11]:
r=np.round(model.predict(x_test[:15]))
print(np.squeeze(r))
print(y_test[:15])


[0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1.]
[0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1.]


In [12]:
print('the test data size:',x_test.shape[0])
%timeit prediction=model.predict(x_test)

the test data size: 100
18.4 ms ± 72.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
model.evaluate(x_test,y_test)

4/4 [==============================] - 0s 1ms/step - loss: 0.0103 - sparse_categorical_accuracy: 0.4500 - binary_accuracy: 0.9900


[0.01029666792601347, 0.44999998807907104, 0.9900000095367432]

As we can see, to predict 100 samples this models takes less than 20 ms with a very high accuracy.